In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras import Input
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/LS - Project/KCDH2024_Training_GroundTruth.xlsx')


In [28]:
labels_dict = {}

label_columns = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
df[label_columns] = df[label_columns].apply(pd.to_numeric,errors='coerce')
for column in label_columns:
    df[column] = pd.to_numeric(df[column])
for index, row in df.iterrows():
    image_name = row['image']
    count=0
    for l in label_columns:
      count+=1
      if row[l] == 1:
        label_index = count
        break
    labels_dict[image_name] = str(label_index)

In [15]:
base_direc_train = './drive/MyDrive/LS - Project/Train_Images'
for grp in label_columns:
  os.makedirs(os.path.join(base_direc_train, grp), exist_ok=True)
images_folder_path = './drive/MyDrive/LS - Project/HAM10000_images_train'
for image_name, label in labels_dict.items():
  source_path = os.path.join(images_folder_path, image_name + '.jpg')
  destination_path = os.path.join(base_direc_train, str(label), image_name + '.jpg')
  if os.path.exists(source_path):
    os.rename(source_path,destination_path)


In [57]:
data=tf.keras.preprocessing.image_dataset_from_directory(base_direc_train,image_size=(128, 128),label_mode = "int")
data=data.map(lambda x,y : (x / 255.0, y))
x_train = data;


Found 9305 files belonging to 7 classes.


In [29]:
df_2 = pd.read_excel('/content/drive/MyDrive/LS - Project/KCDH2024_Test_GroundTruth.xlsx')

In [35]:
labels_dict_2={}
df_2[label_columns] = df_2[label_columns].apply(pd.to_numeric,errors='coerce')
for index, row in df_2.iterrows():
    image_name = row['image']
    for l in label_columns:
      if row[l] == 1:
        label_index = l
        break
    labels_dict_2[image_name] = str(label_index)


In [38]:
base_direc_test = './drive/MyDrive/LS - Project/Test_Images'
for grp in label_columns:
  os.makedirs(os.path.join(base_direc_test, grp), exist_ok=True)
images_folder_path_test = '/content/drive/MyDrive/LS - Project/KCDH2024_Test_Input/KCDH2024_Test_Input'
for image_name, label in labels_dict_2.items():
  source_path = os.path.join(images_folder_path_test, image_name + '.jpg')
  destination_path = os.path.join(base_direc_test, label, image_name + '.jpg')
  if os.path.exists(source_path):
    os.rename(source_path,destination_path)

In [39]:
data_2= tf.keras.preprocessing.image_dataset_from_directory(base_direc_test,image_size=(128, 128),label_mode = "int")
data_2=data_2.map(lambda x,y : (x / 255.0, y))
x_test = data_2;

Found 1350 files belonging to 7 classes.


In [56]:
model =tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128,128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='softmax')
])

In [51]:
class MeanAveragePrecision(tf.keras.metrics.Metric):
    def __init__(self, name='mean_average_precision', **kwargs):
        super(MeanAveragePrecision, self).__init__(name=name, **kwargs)
        self.precisions = self.add_weight(name='precisions', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32).numpy()
        y_pred = tf.cast(y_pred, tf.float32).numpy()

        num_classes = y_pred.shape[1]
        aps = []

        for i in range(num_classes):
            true_labels = (y_true == i).astype(int)
            pred_scores = y_pred[:, i]

            sorted_indices = np.argsort(-pred_scores)
            true_labels_sorted = true_labels[sorted_indices]

            cumsum = np.cumsum(true_labels_sorted)
            precision = cumsum / (np.arange(len(true_labels_sorted)) + 1)
            recall = cumsum / np.sum(true_labels_sorted)

            ap = np.sum(precision * true_labels_sorted) / np.sum(true_labels_sorted)
            aps.append(ap)

        mean_ap = np.mean(aps)
        self.precisions.assign(mean_ap)
        self.count.assign_add(1)

    def result(self):
        return self.precisions

    def reset_states(self):
        self.precisions.assign(0)
        self.count.assign(0)

In [59]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),run_eagerly=True,
              loss = tf.keras.losses.sparse_categorical_crossentropy, # use tf.keras.losses._______(from_logits = False)
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.F1Score(),MeanAveragePrecision()])

In [61]:
model.fit(x_train,epochs = 12,batch_size=32)
results=model.evaluate(x_test)
loss = results[0]
accuracy = results[1]
auc = results[2]
f1_score = results[3]

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
print(f"AUC: {auc}")
print(f"F1 Score: {f1_score}")

Epoch 1/12


InvalidArgumentError: {{function_node __wrapped__SparseSoftmaxCrossEntropyWithLogits_device_/job:localhost/replica:0/task:0/device:CPU:0}} Received a label value of 5 which is outside the valid range of [0, 1).  Label values: 5 4 2 2 5 5 5 5 3 1 5 5 5 2 5 5 5 5 5 5 5 5 5 5 5 1 5 1 2 5 4 0 [Op:SparseSoftmaxCrossEntropyWithLogits] name: 